# Searching for meaning in our ML

Understanding the particular code details of this notebook is secondary!  Your first goal should be to understand that there are techniques and tools for understanding your model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets

from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import DecisionBoundaryDisplay

We'll use an artificial dataset of three clusters using Scikit-Learn's `make_blobs`.

In [ ]:
X, y = make_blobs(n_samples=300, centers=3, random_state=42)

In [ ]:
X.shape

In [ ]:
ycolors = ['red','green','blue']

In [ ]:
plt.scatter(X[:,0],
         X[:,1],
         c=[ycolors[i] for i in y])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

In [ ]:
# Look at the feature coefficients
print('intercepts:\n',lr_model.intercept_)
print('coefficients:\n',lr_model.coef_)

In [ ]:
# Look at the boundaries of predictions in feature space
DecisionBoundaryDisplay.from_estimator(lr_model, 
                                       X,
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

plt.scatter(X[:,0], X[:,1], c=y);

The model equation gives us direct access to these boundaries:

In [ ]:
def mkgrid(a=0):

    plt.scatter(X[:,0], X[:,1], c=y);

    # Create a grid of points
    xbnd = np.linspace(-11, 8, 100)
    ybnd = np.linspace(-11, 13, 100)
    Xbnd, Ybnd = np.meshgrid(xbnd, ybnd)

    # calculate softmax values across the grid
    # and assign them into Z for visualization
    Z = np.zeros((100,100))
    for i in range(100):
        for j in range(100):
            denominator = 0
            for k in range(3):
              denominator += np.exp((lr_model.intercept_[k] +
                                     xbnd[i] * lr_model.coef_[k,0] +
                                     ybnd[j] * lr_model.coef_[k,1]))
            probs = np.exp((lr_model.intercept_[a] +
                            xbnd[i] * lr_model.coef_[a,0] +
                            ybnd[j] * lr_model.coef_[a,1])) / denominator
            Z[i,j] = probs

    # Plot the grid points
    plt.pcolormesh(Xbnd, Ybnd, Z.T, alpha=0.4, cmap='seismic', vmin=0.0, vmax=1.0)
    plt.colorbar()
    plt.show()

ipywidgets.interact(mkgrid, a=(0,2));

In [ ]:
# Print classification report
y_pred = lr_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Look at the confusion matrix
print(confusion_matrix(y_test, y_pred))

## Partial Dependence Plots
These plots show the marginal effect of varying a given feature while keeping all other features constant.

In [ ]:
features_to_plot = [0, 1, (0, 1)]
PartialDependenceDisplay.from_estimator(lr_model, 
                                        X_train, 
                                        features_to_plot,
                                        target=2,
                                        grid_resolution=50);

## LIME
Local Interpretable Model-agnostic Explanations.  LIME takes an instance and generates perturbed samples around it, fits a local interpretable model (like a linear model), and quantifies the contribution of features in this local neighborhood.

In [ ]:
!pip install lime

In [ ]:
from lime import lime_tabular

In [ ]:
# LIME explanation for a single instance
explainer = lime_tabular.LimeTabularExplainer(X_train)

# Choose a random instance for explanation
random_instance_index = np.random.randint(0, len(X_test))
instance_to_explain = X_test[random_instance_index]
print('Instance coordinates:',instance_to_explain)

# Explain the prediction using LIME
explanation = explainer.explain_instance(instance_to_explain, 
                                         lr_model.predict_proba,
                                         top_labels=3)

# Display the explanation
explanation.show_in_notebook()

In [ ]:
def limepoint():
    
    # LIME explanation for a single instance
    explainer = lime_tabular.LimeTabularExplainer(X_train)

    # Choose a random instance for explanation
    random_instance_index = np.random.randint(0, len(X_test))
    instance_to_explain = X_test[random_instance_index]
    print('Instance coordinates:',instance_to_explain)

    # Explain the prediction using LIME
    explanation = explainer.explain_instance(instance_to_explain, 
                                             lr_model.predict_proba,
                                             top_labels=3)

    # Display the explanation
    explanation.show_in_notebook()

    DecisionBoundaryDisplay.from_estimator(lr_model, 
                                           X,
                                           response_method="predict",
                                           cmap="RdBu", 
                                           alpha=0.5
    )
    plt.scatter(X[:,0],
                X[:,1],
                c=[ycolors[i] for i in y])
    plt.scatter(instance_to_explain[0],
                instance_to_explain[1],
                c='black',s=70)
    

In [ ]:
limepoint()

## SHAP
SHapley Additive exPlanations is a "game theoretic approach to explain the output of any machine learning model. It connects optimal credit allocation with local explanations using the classic Shapley values from game theory and their related extensions." -- see https://github.com/shap/shap and [linked papers](https://github.com/shap/shap?tab=readme-ov-file#citations)

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
explainer = shap.LinearExplainer(lr_model, X_train)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)

# Applying this back to handwritten digits

In [ ]:
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.model_selection
import sklearn.metrics

In [ ]:
d = sklearn.datasets.load_digits()

x = d.data
y = d.target

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y)

scaler = sklearn.preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
import sklearn.linear_model
lr_classifier = sklearn.linear_model.LogisticRegression()

In [ ]:
lr_classifier.fit(x_scaled, y_train)

In [ ]:
y_pred = lr_classifier.predict(x_test_scaled)

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
# LIME explanation for a single instance
explainer = lime_tabular.LimeTabularExplainer(x_scaled)

# Choose a random instance for explanation
random_instance_index = np.random.randint(0, len(x_test_scaled))
instance_to_explain = x_test_scaled[random_instance_index]
plt.imshow(instance_to_explain.reshape(8,8),cmap='binary')

# Explain the prediction using LIME
explanation = explainer.explain_instance(instance_to_explain, 
                                         lr_classifier.predict_proba,
                                         top_labels=3)

# Display the explanation
explanation.show_in_notebook()

In [ ]:
explainer = shap.LinearExplainer(lr_classifier, x_scaled)
shap_values = explainer.shap_values(x_test_scaled)

In [ ]:
shap.summary_plot(shap_values, x_test_scaled)

In [ ]:
def whichfeature(feature_num = 43):
    checkimage = np.zeros(64)
    checkimage[feature_num] = 1
    plt.imshow(checkimage.reshape(8,8), cmap='binary')
    
ipywidgets.interact(whichfeature, feature_num=(0,63));

In [ ]:
features_to_plot = [43, 21, (43, 21)]
PartialDependenceDisplay.from_estimator(lr_classifier, 
                                        x_scaled, 
                                        features_to_plot,
                                        target=2,
                                        grid_resolution=50);